In [1]:
import torch

In [2]:
import torch.nn as nn

In [3]:
import torch.nn.functional as F

In [4]:
import os

In [5]:
import requests

# 1 获取数据集-由于无法访问huggingface手动上传

In [7]:
# todo nothing

# 2 读取sales_textbook.txt文件

In [12]:
with open('trans_instance.txt','r') as f:
    text = f.read()

In [13]:
text[0:1000]

'小沈阳江西演唱会邀请，\n明星刀郎的歌火遍大江南北\n2002年的第一场雪比2001年来得更\nLLM张老师的粉丝全是正能量'

# 3 引入tiktoken 将文字token化

In [14]:
import tiktoken

ModuleNotFoundError: No module named 'tiktoken'